In [1]:
import pandas as pd
import numpy as np
import datetime
from calendar import monthrange
import calendar
from datetime import timedelta,datetime
from dateutil.relativedelta import relativedelta

In [2]:
#upload_lines=pd.read_csv("D:\\office\\castlight\\standalone\\upload_line.csv")
upload_lines=pd.read_excel("D:\\vikas\\office\\castlight\\standalone\\upload_v1.xlsx")
ssp=pd.read_csv("D:\\vikas\\office\\castlight\\standalone\\ssp.csv")

In [3]:
upload_lines=upload_lines[upload_lines['Line Type']=='SO']

In [4]:
# considering only standalone cases
s_alone_upload_line = upload_lines[upload_lines['Allocation Type']=='Standalone']

In [5]:
upload_lines = s_alone_upload_line

In [6]:

unique_upload_date = s_alone_upload_line['Upload Month'].unique()
if len(unique_upload_date) ==1 :
    upload_date = s_alone_upload_line['Upload Month'][0]
upload_date
unique_upload_date

array(['2015-01-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [7]:
upload_lines['Next Start Date']=upload_lines.sort_values(['Item ID','Contract Start Date','Contract End Date','User Count']).groupby(['Item ID'])[['Contract Start Date']].shift()
upload_lines['Next End Date']=upload_lines.sort_values(['Item ID','Contract Start Date','Contract End Date','User Count']).groupby(['Item ID'])[['Contract End Date']].shift()

upload_lines['Rank']=upload_lines.sort_values(['Item ID','Contract Start Date','Contract End Date','User Count']).groupby(['Item ID']).cumcount() + 1
upload_lines['Is Overlap']=((upload_lines['Next Start Date'] <= upload_lines['Contract Start Date']) & (upload_lines['Next End Date'] >= upload_lines['Contract End Date']))



In [8]:
def get_allocation_durations(start_date,end_date):
    current = start_date
    result = [current]

    current = current.replace(day=1)
    while current <= end_date:
        current += timedelta(days=32)
        current = current.replace(day=1)
        result.append(datetime(current.year, current.month, 1).date())
    
    durations= []
    for curr in result[:-1]:
        curr_range = calendar.monthrange(curr.year, curr.month)
        curr_duration = (curr_range[1] - curr.day)+1
        if ((curr.month == end_date.month) & (curr.year == end_date.year)):
            durations.append(end_date.day)
        else:
            durations.append(curr_duration)
    return result[:-1],durations

In [9]:
def get_month_difference(start_date,end_date):
    start = datetime(start_date.year, start_date.month, 1)
    end_date+= timedelta(days=1)
    end = datetime(end_date.year, end_date.month, calendar.monthrange(end_date.year, end_date.month)[1])
    x=1-round(end_date.day/calendar.monthrange(end_date.year, end_date.month)[1],2)
    y=round(start_date.day/calendar.monthrange(start_date.year,start_date.month)[1],2)
    return relativedelta(end,start).years*12 + relativedelta(end,start).months -x - y + 1

def get_ratable_revenue(ext_ssp_price,start_date,end_date):
    return ext_ssp_price/(abs((end_date - start_date).days)+1)

def month_year_iter( start_month, start_year, end_month, end_year ):
    ym_start= 12*start_year + start_month - 1
    ym_end= 12*end_year + end_month - 1
    for ym in range( ym_start, ym_end ):
        y, m = divmod( ym, 12 )
        yield y, m+1

In [10]:
upload_lines['Customer Number']=upload_lines['Customer Number'].astype(str)

In [11]:
def perform_standalone_allocation(data,rank):
    data['Allocation Start Date']=data['Contract Start Date'].apply(lambda x : x.date()) 
    data['Allocation End Date']=data['Contract End Date'].apply(lambda x : x.date())
    data['Allocation Term']=data.apply(lambda x: get_month_difference(x['Allocation Start Date'],x['Allocation End Date']), axis=1)
    
    for index, row in data.iterrows():
        if('implementation' in row['Product'].lower() or 'communications standard' in row['Product'].lower()):
            data.loc[data.index[index],'Allocation Term']=1
    
    data['FV Amount']=data['SSP']*data['Allocation Term']*data['User Count']
    data['EXT SSP Price']=data['FV Amount']*data['Contract Value'].sum()/data['FV Amount'].sum()
    data['Flag']=0
    
    data['Allocation Id']=data['Customer Number'] + '-' + data['Sales Order Num'] + '-' + str('{0:0>2}'.format(rank)) + '-' + str('{0:0>2}'.format(data['Flag'].max()+1))
    print(data['Allocation Id'])
   
    data['Allocation Contract Value']=data['Contract Value']
    data['Revenue Start Date']=data['Allocation Start Date']
    data['Revenue End Date']=data['Allocation End Date']
    data['Allocation Start Date']=data['Contract Start Date'].apply(lambda x : x.date()) 
    data['Allocation End Date']=data['Contract End Date'].apply(lambda x : x.date())
    
    data['Ratable Allocation Id'] = data.apply(lambda x: str(x['Allocation Id']) + "-" + str('{0:0>2}'.format(x.name +1)), axis = 1)
    data['Ratable Revenue']=data.apply(lambda x: get_ratable_revenue(x['EXT SSP Price'], x['Revenue Start Date'],x['Revenue End Date']), axis=1)
    
    data['Reallocation Month']=upload_date.date()
    
    export_columns=['Allocation Id','Ratable Allocation Id','Upload Month','Sales Order Num','Sales Order Line ID','Orig SO Line ID','Reallocation Month','Item ID','SSP','SSP Term','User Count','Unit Sell Price','Contract Start Date','Contract End Date','Contract Value','Allocation Start Date','Allocation End Date','Allocation Term','Allocation Contract Value','Revenue Start Date','Revenue End Date','FV Amount','EXT SSP Price','Ratable Revenue','Flag']
    
    monthly_allocation = pd.DataFrame(columns=['ra_id','item','month','year','revenue','flag']) 
    for index, row in data.iterrows():
        months, durations=get_allocation_durations(data.loc[data.index[index], 'Revenue Start Date'],data.loc[data.index[index], 'Revenue End Date'])
        for idx, val in enumerate(months):
#         for value in month_year_iter(data.loc[data.index[index], 'Revenue Start Date'].month,data.loc[data.index[index], 'Revenue Start Date'].year,data.loc[data.index[index], 'Revenue End Date'].month,data.loc[data.index[index], 'Revenue End Date'].year):  
            i=monthly_allocation.shape[0]
            monthly_allocation.loc[i,'item']=row['Item ID']
            monthly_allocation.loc[i,'month']=months[idx].month
            monthly_allocation.loc[i,'year']=months[idx].year
            monthly_allocation.loc[i,'revenue']=durations[idx]*row['Ratable Revenue']
            monthly_allocation.loc[i,'ra_id']=row['Ratable Allocation Id']
            monthly_allocation.loc[i,'flag']=0
        
    
    return data[export_columns], monthly_allocation

    
    
    

In [12]:
if upload_lines['Item ID'].count()==upload_lines['Item ID'].nunique():
    alloc, m_alloc=perform_standalone_allocation(upload_lines,1)
elif upload_lines['Is Overlap'].sum() > 0:
    alloc, m_alloc=perform_standalone_allocation(upload_lines,1)
else:
    for rank in upload_lines['Rank'].unique():
        df=upload_lines[upload_lines['Rank']==rank]
        df=df.reset_index(drop=True)
        if rank == 1:
            alloc, m_alloc=perform_standalone_allocation(df,rank)
        else:
            alloc1,m_alloc1=perform_standalone_allocation(df,rank)
            alloc=alloc.append(alloc1)
            m_alloc=m_alloc.append(m_alloc1)

0    21-21-1021-01-01
1    21-21-1021-01-01
2    21-21-1021-01-01
3    21-21-1021-01-01
4    21-21-1021-01-01
Name: Allocation Id, dtype: object
0    21-21-1021-02-01
1    21-21-1021-02-01
Name: Allocation Id, dtype: object
0    21-21-1021-03-01
1    21-21-1021-03-01
Name: Allocation Id, dtype: object


In [14]:
alloc.to_csv("D:\\vikas\\office\\castlight_v3\\generated_summary\\from_standalone\\allocation_summary_standalone_09_06.csv",index=False)

m_alloc.to_csv("D:\\vikas\\office\\castlight_v3\\generated_summary\\from_standalone\\month_allocation_summary_standalone_09_06.csv",index=False)

In [74]:
alloc


,Allocation Id,Ratable Allocation Id,Upload Month,Sales Order Num,Sales Order Line ID,Orig SO Line ID,Reallocation Month,Item ID,SSP,SSP Term,...,Allocation Start Date,Allocation End Date,Allocation Term,Allocation Contract Value,Revenue Start Date,Revenue End Date,FV Amount,EXT SSP Price,Ratable Revenue,Flag
0,21-21-1021-01-01,21-21-1021-01-01-01,2015-01-01,21-1021,21-1021-1,NaN,2015-01-01,345,2.15,12,...,2015-04-01,2016-03-31,12.0,196800.0,2015-04-01,2016-03-31,258000.0,196063.100437,535.691531,0
1,21-21-1021-01-01,21-21-1021-01-01-02,2015-01-01,21-1021,21-1021-2,NaN,2015-01-01,351,0.70,10,...,2015-06-01,2016-03-31,10.0,55000.0,2015-06-01,2016-03-31,70000.0,53195.414847,174.411196,0
2,21-21-1021-01-01,21-21-1021-01-01-03,2015-01-01,21-1021,21-1021-5,NaN,2015-01-01,366,50000.00,1,...,2015-04-01,2016-03-31,1.0,27500.0,2015-04-01,2016-03-31,50000.0,37996.724891,103.816188,0
3,21-21-1021-01-01,21-21-1021-01-01-04,2015-01-01,21-1021,21-1021-6,NaN,2015-01-01,371,30000.00,1,...,2015-06-01,2016-03-31,1.0,16500.0,2015-06-01,2016-03-31,30000.0,22798.034934,74.747656,0
4,21-21-1021-01-01,21-21-1021-01-01-05,2015-01-01,21-1021,21-1021-9,NaN,2015-01-01,629,50000.00,1,...,2015-04-01,2016-03-31,1.0,52250.0,2015-04-01,2016-03-31,50000.0,37996.724891,103.816188,0
0,21-21-1021-02-01,21-21-1021-02-01-01,2015-01-01,21-1021,21-1021-10,NaN,2015-01-01,345,2.15,12,...,2016-04-01,2017-03-31,12.0,196800.0,2016-04-01,2017-03-31,258000.0,198252.631579,543.157895,0
1,21-21-1021-02-01,21-21-1021-02-01-02,2015-01-01,21-1021,21-1021-11,NaN,2015-01-01,351,0.70,12,...,2016-04-01,2017-03-31,12.0,66000.0,2016-04-01,2017-03-31,84000.0,64547.368421,176.842105,0
0,21-21-1021-03-01,21-21-1021-03-01-01,2015-01-01,21-1021,21-1021-12,NaN,2015-01-01,345,2.15,12,...,2017-04-01,2018-03-31,12.0,196800.0,2017-04-01,2018-03-31,258000.0,198252.631579,543.157895,0
1,21-21-1021-03-01,21-21-1021-03-01-02,2015-01-01,21-1021,21-1021-13,NaN,2015-01-01,351,0.70,12,...,2017-04-01,2018-03-31,12.0,66000.0,2017-04-01,2018-03-31,84000.0,64547.368421,176.842105,0


In [83]:
alloc[['Item ID','Allocation Contract Value','Contract Value','SSP Term','Allocation Term','FV Amount','EXT SSP Price','Ratable Revenue']]

,Item ID,Allocation Contract Value,Contract Value,SSP Term,Allocation Term,FV Amount,EXT SSP Price,Ratable Revenue
0,345,590400.0,590400.0,36,12.0,258000.0,611491.463415,1675.319078
1,351,187000.0,187000.0,34,10.0,70000.0,165908.536585,545.751765
